In [3]:
import pandas as pd
import numpy as np
import pprint
import json
pp = pprint.PrettyPrinter(indent=4)
pd.options.display.max_colwidth = 100

In [4]:
def df_mean(path, apply_to_df):
    df_mean = pd.read_csv(path)
    df_mean = apply_to_df(df_mean)
        
    if 'nb_movies_not_in_train' in df_mean:
        df_mean = df_mean.drop(['nb_movies_not_in_train'],axis=1)
    if 'nb_test_ratings' in df_mean:
        df_mean = df_mean.drop(['nb_test_ratings'],axis=1)
    if 'nb_train_ratings' in df_mean:
        df_mean = df_mean.drop(['nb_train_ratings'],axis=1)
    if 'user_id' in df_mean:
        df_mean = df_mean.drop(['user_id'],axis=1)
    
    df_mean = df_mean.mean(0).to_frame().T
    df_mean['path'] = path
    
    return df_mean
    

def get_df_results(results, max_train_ratings=None):
    if not max_train_ratings:
        apply_to_df = lambda df: df[df['nb_test_ratings'] > 0]
    else:
        apply_to_df = lambda df: df[(df['nb_test_ratings'] > 0) & (df['nb_train_ratings'] <= max_train_ratings)]
        
    result_map = map(lambda path: df_mean(path, apply_to_df), results)
    return reduce(lambda x, y: x.append(y), result_map)

## BEST ML-100k - 0.7 train / 0.3 test - sparse item

In [5]:
user_metrics = [
'../metrics/slim/2016-05-31_15.17.43_slim_ml-100k_e5_tt-0.7_task-16_user-metrics.csv',
'../metrics/slim/2016-06-22_14.07.36_slim_ml-100k_e5_tt-0.7_train-2_task-0_user-metrics.csv',
'../metrics/slim/2016-06-22_14.08.21_slim_ml-100k_e5_tt-0.7_train-3_task-0_user-metrics.csv',
'../metrics/slim/2016-06-22_14.09.49_slim_ml-100k_e5_tt-0.7_train-4_task-0_user-metrics.csv',
'../metrics/slim/2016-06-22_14.10.56_slim_ml-100k_e5_tt-0.7_train-5_task-0_user-metrics.csv',
]

In [6]:
df_cv_user_metrics = get_df_results(user_metrics)
df_cv_user_metrics

,auc,avg_precision,f1,fcp,precision_at_20,recall_at_20,reciprocal_rank,spearman_rank_corr,path
0,0.914040,0.262888,0.238686,0.716026,0.233192,0.375544,0.591272,0.192209,../metrics/slim/2016-05-31_15.17.43_slim_ml-100k_e5_tt-0.7_task-16_user-metrics.csv
0,0.913664,0.267736,0.238506,0.713535,0.234464,0.377523,0.605176,0.191109,../metrics/slim/2016-06-22_14.07.36_slim_ml-100k_e5_tt-0.7_train-2_task-0_user-metrics.csv
0,0.914962,0.271273,0.243517,0.713476,0.237911,0.379707,0.597825,0.193866,../metrics/slim/2016-06-22_14.08.21_slim_ml-100k_e5_tt-0.7_train-3_task-0_user-metrics.csv
0,0.913825,0.266452,0.238997,0.707779,0.233086,0.375192,0.600695,0.183056,../metrics/slim/2016-06-22_14.09.49_slim_ml-100k_e5_tt-0.7_train-4_task-0_user-metrics.csv
0,0.915789,0.265468,0.240869,0.709363,0.235790,0.379723,0.601344,0.179654,../metrics/slim/2016-06-22_14.10.56_slim_ml-100k_e5_tt-0.7_train-5_task-0_user-metrics.csv


In [7]:
df_cv_user_metrics.mean(0)

auc                   0.914456
avg_precision         0.266763
f1                    0.240115
fcp                   0.712036
precision_at_20       0.234889
recall_at_20          0.377538
reciprocal_rank       0.599262
spearman_rank_corr    0.187979
dtype: float64

In [8]:
with open('../models/slim/2016-05-31_15.17.43_slim_ml-100k_e5_tt-0.7_task-16_config.json') as f:
    pp.pprint(json.loads(f.read()))

{   u'alpha': 0.00303,
    u'binarize': True,
    u'binarize_neg': 0,
    u'binarize_pos': 1,
    u'binarize_threshold': 1,
    u'eval_in_parallel': False,
    u'experiment_name': u'slim_ml-100k_e5_tt-0.7_task-16',
    u'fit_intercept': False,
    u'hit_threshold': 4,
    u'ignore_negative_weights': True,
    u'l1_ratio': 0.9900990099009901,
    u'l1_reg': 0.003,
    u'l2_reg': 3e-05,
    u'metrics_save_dir': u'metrics/slim',
    u'model_save_dir': u'models/slim',
    u'nb_epochs': 5,
    u'precision_recall_at_n': 20,
    u'ratings_path': u'data/splits/ml-100k/ratings.csv',
    u'run_eval': True,
    u'run_movie_metrics': False,
    u'sparse_item': True,
    u'test': True,
    u'test_path': u'data/splits/ml-100k/sparse-item/0.7-test.csv',
    u'top_n_predictions': 100,
    u'train_path': u'data/splits/ml-100k/sparse-item/0.7-train.csv',
    u'train_test_split': 0.7,
    u'val': False,
    u'verbose': 0}


In [9]:
movie_metrics = [
'../metrics/slim/2016-05-31_15.17.43_slim_ml-100k_e5_tt-0.7_task-16_movie-metrics.csv',
'../metrics/slim/2016-06-22_14.07.36_slim_ml-100k_e5_tt-0.7_train-2_task-0_movie-metrics.csv',
'../metrics/slim/2016-06-22_14.08.21_slim_ml-100k_e5_tt-0.7_train-3_task-0_movie-metrics.csv',
'../metrics/slim/2016-06-22_14.09.49_slim_ml-100k_e5_tt-0.7_train-4_task-0_movie-metrics.csv',
'../metrics/slim/2016-06-22_14.10.56_slim_ml-100k_e5_tt-0.7_train-5_task-0_movie-metrics.csv',
]

In [10]:
df_cv_movie_metrics = get_df_results(movie_metrics)
df_cv_movie_metrics

,auc,movie_id,nb_times_in_top_n_predictions,path
0,0.854889,708.548841,78.062914,../metrics/slim/2016-05-31_15.17.43_slim_ml-100k_e5_tt-0.7_task-16_movie-metrics.csv
0,0.849859,708.548841,78.062914,../metrics/slim/2016-06-22_14.07.36_slim_ml-100k_e5_tt-0.7_train-2_task-0_movie-metrics.csv
0,0.854410,708.548841,78.062914,../metrics/slim/2016-06-22_14.08.21_slim_ml-100k_e5_tt-0.7_train-3_task-0_movie-metrics.csv
0,0.847583,708.548841,78.062914,../metrics/slim/2016-06-22_14.09.49_slim_ml-100k_e5_tt-0.7_train-4_task-0_movie-metrics.csv
0,0.853314,708.548841,78.062914,../metrics/slim/2016-06-22_14.10.56_slim_ml-100k_e5_tt-0.7_train-5_task-0_movie-metrics.csv


In [11]:
df_cv_movie_metrics.mean(0)

auc                                0.852011
movie_id                         708.548841
nb_times_in_top_n_predictions     78.062914
dtype: float64

## BEST ML-1m - 0.7 train / 0.3 test - sparse item

In [19]:
user_metrics = [
'../metrics/slim/2016-05-31_15.25.26_slim_ml-1m_e5_tt-0.7_task-10_user-metrics.csv',
'../metrics/slim/2016-06-22_14.44.11_slim_ml-1m_e5_tt-0.7_train-2_task-0_user-metrics.csv',
'../metrics/slim/2016-06-22_14.44.37_slim_ml-1m_e5_tt-0.7_train-3_task-0_user-metrics.csv',
'../metrics/slim/2016-06-22_14.45.33_slim_ml-1m_e5_tt-0.7_train-4_task-0_user-metrics.csv',
'../metrics/slim/2016-06-22_14.46.16_slim_ml-1m_e5_tt-0.7_train-5_task-0_user-metrics.csv',
]

In [20]:
df_cv_user_metrics = get_df_results(user_metrics)
df_cv_user_metrics

,auc,avg_precision,f1,fcp,precision_at_20,recall_at_20,reciprocal_rank,spearman_rank_corr,path
0,0.912640,0.251286,0.233695,0.731258,0.286267,0.289717,0.647693,0.233961,../metrics/slim/2016-05-31_15.25.26_slim_ml-1m_e5_tt-0.7_task-10_user-metrics.csv
0,0.912689,0.252171,0.232878,0.731217,0.285497,0.288505,0.642786,0.235151,../metrics/slim/2016-06-22_14.44.11_slim_ml-1m_e5_tt-0.7_train-2_task-0_user-metrics.csv
0,0.912005,0.252220,0.233696,0.730978,0.286333,0.287568,0.645496,0.232977,../metrics/slim/2016-06-22_14.44.37_slim_ml-1m_e5_tt-0.7_train-3_task-0_user-metrics.csv
0,0.912750,0.253179,0.232055,0.729666,0.285174,0.286280,0.653019,0.227353,../metrics/slim/2016-06-22_14.45.33_slim_ml-1m_e5_tt-0.7_train-4_task-0_user-metrics.csv
0,0.911612,0.250127,0.232341,0.730427,0.286060,0.286146,0.643594,0.231489,../metrics/slim/2016-06-22_14.46.16_slim_ml-1m_e5_tt-0.7_train-5_task-0_user-metrics.csv


In [21]:
df_cv_user_metrics.mean(0)

auc                   0.912339
avg_precision         0.251796
f1                    0.232933
fcp                   0.730709
precision_at_20       0.285866
recall_at_20          0.287643
reciprocal_rank       0.646518
spearman_rank_corr    0.232186
dtype: float64

In [15]:
with open('../models/slim/2016-05-31_15.25.26_slim_ml-1m_e5_tt-0.7_task-10_config.json') as f:
    pp.pprint(json.loads(f.read()))

{   u'alpha': 0.004,
    u'binarize': True,
    u'binarize_neg': 0,
    u'binarize_pos': 1,
    u'binarize_threshold': 1,
    u'eval_in_parallel': False,
    u'experiment_name': u'slim_ml-1m_e5_tt-0.7_task-10',
    u'fit_intercept': True,
    u'hit_threshold': 4,
    u'ignore_negative_weights': True,
    u'l1_ratio': 0.25,
    u'l1_reg': 0.001,
    u'l2_reg': 0.003,
    u'metrics_save_dir': u'metrics/slim',
    u'model_save_dir': u'models/slim',
    u'nb_epochs': 5,
    u'precision_recall_at_n': 20,
    u'ratings_path': u'data/splits/ml-1m/ratings.csv',
    u'run_eval': True,
    u'run_movie_metrics': False,
    u'sparse_item': True,
    u'test': True,
    u'test_path': u'data/splits/ml-1m/sparse-item/0.7-test.csv',
    u'top_n_predictions': 100,
    u'train_path': u'data/splits/ml-1m/sparse-item/0.7-train.csv',
    u'train_test_split': 0.7,
    u'val': False,
    u'verbose': 0}


In [22]:
movie_metrics = [
'../metrics/slim/2016-05-31_15.25.26_slim_ml-1m_e5_tt-0.7_task-10_movie-metrics.csv',
'../metrics/slim/2016-06-22_14.44.11_slim_ml-1m_e5_tt-0.7_train-2_task-0_movie-metrics.csv',
'../metrics/slim/2016-06-22_14.44.37_slim_ml-1m_e5_tt-0.7_train-3_task-0_movie-metrics.csv',
'../metrics/slim/2016-06-22_14.45.33_slim_ml-1m_e5_tt-0.7_train-4_task-0_movie-metrics.csv',
'../metrics/slim/2016-06-22_14.46.16_slim_ml-1m_e5_tt-0.7_train-5_task-0_movie-metrics.csv',
]

In [23]:
df_cv_movie_metrics = get_df_results(movie_metrics)
df_cv_movie_metrics

,auc,movie_id,nb_times_in_top_n_predictions,path
0,0.881056,1999.3961,203.093477,../metrics/slim/2016-05-31_15.25.26_slim_ml-1m_e5_tt-0.7_task-10_movie-metrics.csv
0,0.880763,1999.3961,203.093477,../metrics/slim/2016-06-22_14.44.11_slim_ml-1m_e5_tt-0.7_train-2_task-0_movie-metrics.csv
0,0.879559,1999.3961,203.093477,../metrics/slim/2016-06-22_14.44.37_slim_ml-1m_e5_tt-0.7_train-3_task-0_movie-metrics.csv
0,0.879966,1999.3961,203.093477,../metrics/slim/2016-06-22_14.45.33_slim_ml-1m_e5_tt-0.7_train-4_task-0_movie-metrics.csv
0,0.878792,1999.3961,203.093477,../metrics/slim/2016-06-22_14.46.16_slim_ml-1m_e5_tt-0.7_train-5_task-0_movie-metrics.csv


In [24]:
df_cv_movie_metrics.mean(0)

auc                                 0.880027
movie_id                         1999.396100
nb_times_in_top_n_predictions     203.093477
dtype: float64

# Gridsearch

## ML-100k - 0.7 train / 0.3 test - sparse item

#### User Metrics

In [2]:
user_metrics = [
'../metrics/slim/2016-05-31_15.17.38_slim_ml-100k_e3_tt-0.7_task-14_user-metrics.csv',
'../metrics/slim/2016-05-31_15.17.39_slim_ml-100k_e3_tt-0.7_task-3_user-metrics.csv',
'../metrics/slim/2016-05-31_15.17.39_slim_ml-100k_e3_tt-0.7_task-4_user-metrics.csv',
'../metrics/slim/2016-05-31_15.17.40_slim_ml-100k_e3_tt-0.7_task-21_user-metrics.csv',
'../metrics/slim/2016-05-31_15.17.41_slim_ml-100k_e3_tt-0.7_task-10_user-metrics.csv',
'../metrics/slim/2016-05-31_15.17.41_slim_ml-100k_e3_tt-0.7_task-12_user-metrics.csv',
'../metrics/slim/2016-05-31_15.17.42_slim_ml-100k_e3_tt-0.7_task-11_user-metrics.csv',
'../metrics/slim/2016-05-31_15.17.42_slim_ml-100k_e3_tt-0.7_task-23_user-metrics.csv',
'../metrics/slim/2016-05-31_15.17.42_slim_ml-100k_e3_tt-0.7_task-7_user-metrics.csv',
'../metrics/slim/2016-05-31_15.17.43_slim_ml-100k_e5_tt-0.7_task-16_user-metrics.csv',
'../metrics/slim/2016-05-31_15.17.43_slim_ml-100k_e5_tt-0.7_task-22_user-metrics.csv',
'../metrics/slim/2016-05-31_15.17.43_slim_ml-100k_e5_tt-0.7_task-9_user-metrics.csv',
'../metrics/slim/2016-05-31_15.17.46_slim_ml-100k_e5_tt-0.7_task-0_user-metrics.csv',
'../metrics/slim/2016-05-31_15.17.46_slim_ml-100k_e5_tt-0.7_task-13_user-metrics.csv',
'../metrics/slim/2016-05-31_15.17.46_slim_ml-100k_e5_tt-0.7_task-6_user-metrics.csv',
'../metrics/slim/2016-05-31_15.17.47_slim_ml-100k_e5_tt-0.7_task-1_user-metrics.csv',
'../metrics/slim/2016-05-31_15.17.47_slim_ml-100k_e5_tt-0.7_task-2_user-metrics.csv',
'../metrics/slim/2016-05-31_15.17.48_slim_ml-100k_e8_tt-0.7_task-8_user-metrics.csv',
'../metrics/slim/2016-05-31_15.17.51_slim_ml-100k_e10_tt-0.7_task-20_user-metrics.csv',
'../metrics/slim/2016-05-31_15.17.53_slim_ml-100k_e8_tt-0.7_task-19_user-metrics.csv',
'../metrics/slim/2016-05-31_15.17.54_slim_ml-100k_e8_tt-0.7_task-15_user-metrics.csv',
'../metrics/slim/2016-05-31_15.17.57_slim_ml-100k_e10_tt-0.7_task-18_user-metrics.csv',
'../metrics/slim/2016-05-31_15.17.57_slim_ml-100k_e10_tt-0.7_task-5_user-metrics.csv',
'../metrics/slim/2016-05-31_15.17.58_slim_ml-100k_e10_tt-0.7_task-17_user-metrics.csv',
]

In [5]:
df_results = get_df_results(user_metrics)
df_results.sort_values('auc', ascending=False)

,auc,avg_precision,f1,fcp,precision_at_20,recall_at_20,reciprocal_rank,spearman_rank_corr,path
0,0.914040,0.262888,0.238686,0.716026,0.233192,0.375544,0.591272,0.192209,../metrics/slim/2016-05-31_15.17.43_slim_ml-100k_e5_tt-0.7_task-16_user-metrics.csv
0,0.913098,0.263710,0.240781,0.713194,0.235631,0.378372,0.579589,0.185651,../metrics/slim/2016-05-31_15.17.40_slim_ml-100k_e3_tt-0.7_task-21_user-metrics.csv
0,0.911028,0.258207,0.234696,0.715728,0.229162,0.369422,0.574550,0.191514,../metrics/slim/2016-05-31_15.17.41_slim_ml-100k_e3_tt-0.7_task-12_user-metrics.csv
0,0.910040,0.264643,0.238594,0.714940,0.233775,0.375495,0.587817,0.189875,../metrics/slim/2016-05-31_15.17.43_slim_ml-100k_e5_tt-0.7_task-22_user-metrics.csv
0,0.909430,0.264419,0.238393,0.714634,0.233457,0.375381,0.587354,0.189084,../metrics/slim/2016-05-31_15.17.43_slim_ml-100k_e5_tt-0.7_task-9_user-metrics.csv
0,0.908577,0.254513,0.232541,0.715339,0.226882,0.367258,0.571357,0.190660,../metrics/slim/2016-05-31_15.17.42_slim_ml-100k_e3_tt-0.7_task-7_user-metrics.csv
0,0.908572,0.254546,0.232541,0.715358,0.226882,0.367258,0.571330,0.190694,../metrics/slim/2016-05-31_15.17.42_slim_ml-100k_e3_tt-0.7_task-11_user-metrics.csv
0,0.908572,0.255354,0.232385,0.715348,0.226776,0.366941,0.572564,0.190931,../metrics/slim/2016-05-31_15.17.41_slim_ml-100k_e3_tt-0.7_task-10_user-metrics.csv
0,0.906182,0.251422,0.229869,0.715577,0.223913,0.364317,0.568639,0.191878,../metrics/slim/2016-05-31_15.17.46_slim_ml-100k_e5_tt-0.7_task-13_user-metrics.csv
0,0.905768,0.263664,0.239178,0.713250,0.234146,0.375451,0.586188,0.185944,../metrics/slim/2016-05-31_15.17.48_slim_ml-100k_e8_tt-0.7_task-8_user-metrics.csv


In [14]:
# best config
with open('../models/slim/2016-05-31_15.17.43_slim_ml-100k_e5_tt-0.7_task-16_config.json') as f:
    pp.pprint(json.loads(f.read()))

{   u'alpha': 0.00303,
    u'binarize': True,
    u'binarize_neg': 0,
    u'binarize_pos': 1,
    u'binarize_threshold': 1,
    u'eval_in_parallel': False,
    u'experiment_name': u'slim_ml-100k_e5_tt-0.7_task-16',
    u'fit_intercept': False,
    u'hit_threshold': 4,
    u'ignore_negative_weights': True,
    u'l1_ratio': 0.9900990099009901,
    u'l1_reg': 0.003,
    u'l2_reg': 3e-05,
    u'metrics_save_dir': u'metrics/slim',
    u'model_save_dir': u'models/slim',
    u'nb_epochs': 5,
    u'precision_recall_at_n': 20,
    u'ratings_path': u'data/splits/ml-100k/ratings.csv',
    u'run_eval': True,
    u'run_movie_metrics': False,
    u'sparse_item': True,
    u'test': True,
    u'test_path': u'data/splits/ml-100k/sparse-item/0.7-test.csv',
    u'top_n_predictions': 100,
    u'train_path': u'data/splits/ml-100k/sparse-item/0.7-train.csv',
    u'train_test_split': 0.7,
    u'val': False,
    u'verbose': 0}


In [1]:
movie_metrics = [
'../metrics/slim/2016-05-31_15.17.38_slim_ml-100k_e3_tt-0.7_task-14_movie-metrics.csv',
'../metrics/slim/2016-05-31_15.17.39_slim_ml-100k_e3_tt-0.7_task-3_movie-metrics.csv',
'../metrics/slim/2016-05-31_15.17.39_slim_ml-100k_e3_tt-0.7_task-4_movie-metrics.csv',
'../metrics/slim/2016-05-31_15.17.40_slim_ml-100k_e3_tt-0.7_task-21_movie-metrics.csv',
'../metrics/slim/2016-05-31_15.17.41_slim_ml-100k_e3_tt-0.7_task-10_movie-metrics.csv',
'../metrics/slim/2016-05-31_15.17.41_slim_ml-100k_e3_tt-0.7_task-12_movie-metrics.csv',
'../metrics/slim/2016-05-31_15.17.42_slim_ml-100k_e3_tt-0.7_task-11_movie-metrics.csv',
'../metrics/slim/2016-05-31_15.17.42_slim_ml-100k_e3_tt-0.7_task-23_movie-metrics.csv',
'../metrics/slim/2016-05-31_15.17.42_slim_ml-100k_e3_tt-0.7_task-7_movie-metrics.csv',
'../metrics/slim/2016-05-31_15.17.43_slim_ml-100k_e5_tt-0.7_task-16_movie-metrics.csv',
'../metrics/slim/2016-05-31_15.17.43_slim_ml-100k_e5_tt-0.7_task-22_movie-metrics.csv',
'../metrics/slim/2016-05-31_15.17.43_slim_ml-100k_e5_tt-0.7_task-9_movie-metrics.csv',
'../metrics/slim/2016-05-31_15.17.46_slim_ml-100k_e5_tt-0.7_task-0_movie-metrics.csv',
'../metrics/slim/2016-05-31_15.17.46_slim_ml-100k_e5_tt-0.7_task-13_movie-metrics.csv',
'../metrics/slim/2016-05-31_15.17.46_slim_ml-100k_e5_tt-0.7_task-6_movie-metrics.csv',
'../metrics/slim/2016-05-31_15.17.47_slim_ml-100k_e5_tt-0.7_task-1_movie-metrics.csv',
'../metrics/slim/2016-05-31_15.17.47_slim_ml-100k_e5_tt-0.7_task-2_movie-metrics.csv',
'../metrics/slim/2016-05-31_15.17.48_slim_ml-100k_e8_tt-0.7_task-8_movie-metrics.csv',
'../metrics/slim/2016-05-31_15.17.51_slim_ml-100k_e10_tt-0.7_task-20_movie-metrics.csv',
'../metrics/slim/2016-05-31_15.17.53_slim_ml-100k_e8_tt-0.7_task-19_movie-metrics.csv',
'../metrics/slim/2016-05-31_15.17.54_slim_ml-100k_e8_tt-0.7_task-15_movie-metrics.csv',
'../metrics/slim/2016-05-31_15.17.57_slim_ml-100k_e10_tt-0.7_task-18_movie-metrics.csv',
'../metrics/slim/2016-05-31_15.17.57_slim_ml-100k_e10_tt-0.7_task-5_movie-metrics.csv',
'../metrics/slim/2016-05-31_15.17.58_slim_ml-100k_e10_tt-0.7_task-17_movie-metrics.csv',
]

In [6]:
df_movie_metrics = get_df_results(movie_metrics)
df_movie_metrics.sort_values('auc', ascending=False)[['auc', 'path']]

,auc,path
0,0.867189,../metrics/slim/2016-05-31_15.17.41_slim_ml-100k_e3_tt-0.7_task-12_movie-metrics.csv
0,0.856618,../metrics/slim/2016-05-31_15.17.40_slim_ml-100k_e3_tt-0.7_task-21_movie-metrics.csv
0,0.855623,../metrics/slim/2016-05-31_15.17.43_slim_ml-100k_e5_tt-0.7_task-9_movie-metrics.csv
0,0.855444,../metrics/slim/2016-05-31_15.17.41_slim_ml-100k_e3_tt-0.7_task-10_movie-metrics.csv
0,0.855438,../metrics/slim/2016-05-31_15.17.42_slim_ml-100k_e3_tt-0.7_task-7_movie-metrics.csv
0,0.855434,../metrics/slim/2016-05-31_15.17.42_slim_ml-100k_e3_tt-0.7_task-11_movie-metrics.csv
0,0.854922,../metrics/slim/2016-05-31_15.17.43_slim_ml-100k_e5_tt-0.7_task-22_movie-metrics.csv
0,0.854889,../metrics/slim/2016-05-31_15.17.43_slim_ml-100k_e5_tt-0.7_task-16_movie-metrics.csv
0,0.852338,../metrics/slim/2016-05-31_15.17.46_slim_ml-100k_e5_tt-0.7_task-13_movie-metrics.csv
0,0.851824,../metrics/slim/2016-05-31_15.17.51_slim_ml-100k_e10_tt-0.7_task-20_movie-metrics.csv


## ML-1M - 0.7 train / 0.3 test - sparse item

#### User Metrics

In [7]:
user_metrics = [
'../metrics/slim/2016-05-31_15.23.01_slim_ml-1m_e3_tt-0.7_task-21_user-metrics.csv',
'../metrics/slim/2016-05-31_15.23.03_slim_ml-1m_e3_tt-0.7_task-8_user-metrics.csv',
'../metrics/slim/2016-05-31_15.23.23_slim_ml-1m_e3_tt-0.7_task-22_user-metrics.csv',
'../metrics/slim/2016-05-31_15.23.31_slim_ml-1m_e3_tt-0.7_task-23_user-metrics.csv',
'../metrics/slim/2016-05-31_15.23.32_slim_ml-1m_e3_tt-0.7_task-16_user-metrics.csv',
'../metrics/slim/2016-05-31_15.23.44_slim_ml-1m_e3_tt-0.7_task-11_user-metrics.csv',
'../metrics/slim/2016-05-31_15.23.56_slim_ml-1m_e3_tt-0.7_task-2_user-metrics.csv',
'../metrics/slim/2016-05-31_15.24.05_slim_ml-1m_e3_tt-0.7_task-5_user-metrics.csv',
'../metrics/slim/2016-05-31_15.24.22_slim_ml-1m_e5_tt-0.7_task-20_user-metrics.csv',
'../metrics/slim/2016-05-31_15.25.13_slim_ml-1m_e5_tt-0.7_task-6_user-metrics.csv',
'../metrics/slim/2016-05-31_15.25.16_slim_ml-1m_e5_tt-0.7_task-9_user-metrics.csv',
'../metrics/slim/2016-05-31_15.25.26_slim_ml-1m_e5_tt-0.7_task-10_user-metrics.csv',
'../metrics/slim/2016-05-31_15.25.26_slim_ml-1m_e5_tt-0.7_task-7_user-metrics.csv',
'../metrics/slim/2016-05-31_15.25.52_slim_ml-1m_e5_tt-0.7_task-19_user-metrics.csv',
'../metrics/slim/2016-05-31_15.26.01_slim_ml-1m_e8_tt-0.7_task-4_user-metrics.csv',
'../metrics/slim/2016-05-31_15.26.07_slim_ml-1m_e5_tt-0.7_task-0_user-metrics.csv',
'../metrics/slim/2016-05-31_15.26.08_slim_ml-1m_e5_tt-0.7_task-1_user-metrics.csv',
'../metrics/slim/2016-05-31_15.26.29_slim_ml-1m_e5_tt-0.7_task-3_user-metrics.csv',
'../metrics/slim/2016-05-31_15.26.33_slim_ml-1m_e5_tt-0.7_task-18_user-metrics.csv',
'../metrics/slim/2016-05-31_15.27.02_slim_ml-1m_e8_tt-0.7_task-12_user-metrics.csv',
'../metrics/slim/2016-05-31_15.27.32_slim_ml-1m_e8_tt-0.7_task-13_user-metrics.csv',
'../metrics/slim/2016-05-31_15.28.26_slim_ml-1m_e10_tt-0.7_task-15_user-metrics.csv',
'../metrics/slim/2016-05-31_15.28.52_slim_ml-1m_e10_tt-0.7_task-17_user-metrics.csv',
'../metrics/slim/2016-05-31_15.30.39_slim_ml-1m_e10_tt-0.7_task-14_user-metrics.csv',
]

In [8]:
df_results = get_df_results(user_metrics)
df_results.sort_values('auc', ascending=False)

,auc,avg_precision,f1,fcp,precision_at_20,recall_at_20,reciprocal_rank,spearman_rank_corr,path
0,0.912640,0.251286,0.233695,0.731258,0.286267,0.289717,0.647693,0.233961,../metrics/slim/2016-05-31_15.25.26_slim_ml-1m_e5_tt-0.7_task-10_user-metrics.csv
0,0.912340,0.251035,0.233457,0.731137,0.285861,0.289431,0.647407,0.233578,../metrics/slim/2016-05-31_15.25.26_slim_ml-1m_e5_tt-0.7_task-7_user-metrics.csv
0,0.906662,0.251155,0.231786,0.728287,0.285927,0.285271,0.644260,0.224911,../metrics/slim/2016-05-31_15.23.31_slim_ml-1m_e3_tt-0.7_task-23_user-metrics.csv
0,0.906519,0.251047,0.231620,0.728138,0.285728,0.284978,0.643693,0.224368,../metrics/slim/2016-05-31_15.23.23_slim_ml-1m_e3_tt-0.7_task-22_user-metrics.csv
0,0.905546,0.251778,0.232281,0.727542,0.286548,0.285682,0.643783,0.222522,../metrics/slim/2016-05-31_15.23.03_slim_ml-1m_e3_tt-0.7_task-8_user-metrics.csv
0,0.904954,0.241433,0.228100,0.731810,0.278171,0.283907,0.641043,0.235019,../metrics/slim/2016-05-31_15.24.05_slim_ml-1m_e3_tt-0.7_task-5_user-metrics.csv
0,0.904049,0.249911,0.230497,0.726019,0.284859,0.283630,0.641467,0.218412,../metrics/slim/2016-05-31_15.28.26_slim_ml-1m_e10_tt-0.7_task-15_user-metrics.csv
0,0.903893,0.250472,0.230553,0.726333,0.284727,0.283476,0.642647,0.219147,../metrics/slim/2016-05-31_15.26.01_slim_ml-1m_e8_tt-0.7_task-4_user-metrics.csv
0,0.903808,0.250377,0.230443,0.726352,0.284570,0.283354,0.642326,0.219192,../metrics/slim/2016-05-31_15.27.02_slim_ml-1m_e8_tt-0.7_task-12_user-metrics.csv
0,0.901985,0.232596,0.220997,0.731566,0.269478,0.275533,0.630855,0.234607,../metrics/slim/2016-05-31_15.30.39_slim_ml-1m_e10_tt-0.7_task-14_user-metrics.csv


In [18]:
# best config
with open('../models/slim/2016-05-31_15.25.26_slim_ml-1m_e5_tt-0.7_task-10_config.json') as f:
    pp.pprint(json.loads(f.read()))

{   u'alpha': 0.004,
    u'binarize': True,
    u'binarize_neg': 0,
    u'binarize_pos': 1,
    u'binarize_threshold': 1,
    u'eval_in_parallel': False,
    u'experiment_name': u'slim_ml-1m_e5_tt-0.7_task-10',
    u'fit_intercept': True,
    u'hit_threshold': 4,
    u'ignore_negative_weights': True,
    u'l1_ratio': 0.25,
    u'l1_reg': 0.001,
    u'l2_reg': 0.003,
    u'metrics_save_dir': u'metrics/slim',
    u'model_save_dir': u'models/slim',
    u'nb_epochs': 5,
    u'precision_recall_at_n': 20,
    u'ratings_path': u'data/splits/ml-1m/ratings.csv',
    u'run_eval': True,
    u'run_movie_metrics': False,
    u'sparse_item': True,
    u'test': True,
    u'test_path': u'data/splits/ml-1m/sparse-item/0.7-test.csv',
    u'top_n_predictions': 100,
    u'train_path': u'data/splits/ml-1m/sparse-item/0.7-train.csv',
    u'train_test_split': 0.7,
    u'val': False,
    u'verbose': 0}


## ML-100k - 0.56 train/0.14 val/0.3 test

In [2]:
results = [
'../metrics/slim/2016-05-10_03.05.12_slim_ml-100k_e3_tt-0.7_task-0_user-metrics.csv',
'../metrics/slim/2016-05-10_03.05.15_slim_ml-100k_e8_tt-0.7_task-12_user-metrics.csv',
'../metrics/slim/2016-05-10_03.05.26_slim_ml-100k_e10_tt-0.7_task-42_user-metrics.csv',
'../metrics/slim/2016-05-10_03.05.34_slim_ml-100k_e8_tt-0.7_task-30_user-metrics.csv',
'../metrics/slim/2016-05-10_03.05.36_slim_ml-100k_e10_tt-0.7_task-10_user-metrics.csv',
'../metrics/slim/2016-05-10_03.05.37_slim_ml-100k_e3_tt-0.7_task-22_user-metrics.csv',
'../metrics/slim/2016-05-10_03.05.55_slim_ml-100k_e3_tt-0.7_task-2_user-metrics.csv',
'../metrics/slim/2016-05-10_03.05.59_slim_ml-100k_e3_tt-0.7_task-18_user-metrics.csv',
'../metrics/slim/2016-05-10_03.06.00_slim_ml-100k_e8_tt-0.7_task-44_user-metrics.csv',
'../metrics/slim/2016-05-10_03.06.04_slim_ml-100k_e3_tt-0.7_task-24_user-metrics.csv',
'../metrics/slim/2016-05-10_03.06.06_slim_ml-100k_e10_tt-0.7_task-8_user-metrics.csv',
'../metrics/slim/2016-05-10_03.06.08_slim_ml-100k_e3_tt-0.7_task-28_user-metrics.csv',
'../metrics/slim/2016-05-10_03.06.10_slim_ml-100k_e10_tt-0.7_task-40_user-metrics.csv',
'../metrics/slim/2016-05-10_03.06.10_slim_ml-100k_e5_tt-0.7_task-32_user-metrics.csv',
'../metrics/slim/2016-05-10_03.06.12_slim_ml-100k_e3_tt-0.7_task-20_user-metrics.csv',
'../metrics/slim/2016-05-10_03.06.14_slim_ml-100k_e5_tt-0.7_task-38_user-metrics.csv',
'../metrics/slim/2016-05-10_03.06.16_slim_ml-100k_e5_tt-0.7_task-4_user-metrics.csv',
'../metrics/slim/2016-05-10_03.06.18_slim_ml-100k_e10_tt-0.7_task-46_user-metrics.csv',
'../metrics/slim/2016-05-10_03.06.21_slim_ml-100k_e8_tt-0.7_task-34_user-metrics.csv',
'../metrics/slim/2016-05-10_03.06.24_slim_ml-100k_e10_tt-0.7_task-6_user-metrics.csv',
'../metrics/slim/2016-05-10_03.06.25_slim_ml-100k_e5_tt-0.7_task-14_user-metrics.csv',
'../metrics/slim/2016-05-10_03.06.29_slim_ml-100k_e8_tt-0.7_task-26_user-metrics.csv',
'../metrics/slim/2016-05-10_03.06.33_slim_ml-100k_e10_tt-0.7_task-16_user-metrics.csv',
'../metrics/slim/2016-05-10_03.06.40_slim_ml-100k_e8_tt-0.7_task-36_user-metrics.csv',
'../metrics/slim/2016-05-10_03.23.59_slim_ml-100k_e3_tt-0.7_task-1_user-metrics.csv',
'../metrics/slim/2016-05-10_03.24.15_slim_ml-100k_e3_tt-0.7_task-29_user-metrics.csv',
'../metrics/slim/2016-05-10_03.24.17_slim_ml-100k_e5_tt-0.7_task-43_user-metrics.csv',
'../metrics/slim/2016-05-10_03.24.32_slim_ml-100k_e10_tt-0.7_task-3_user-metrics.csv',
'../metrics/slim/2016-05-10_03.24.32_slim_ml-100k_e3_tt-0.7_task-41_user-metrics.csv',
'../metrics/slim/2016-05-10_03.24.33_slim_ml-100k_e5_tt-0.7_task-11_user-metrics.csv',
'../metrics/slim/2016-05-10_03.24.33_slim_ml-100k_e8_tt-0.7_task-23_user-metrics.csv',
'../metrics/slim/2016-05-10_03.24.47_slim_ml-100k_e8_tt-0.7_task-13_user-metrics.csv',
'../metrics/slim/2016-05-10_03.24.53_slim_ml-100k_e8_tt-0.7_task-19_user-metrics.csv',
'../metrics/slim/2016-05-10_03.24.54_slim_ml-100k_e5_tt-0.7_task-45_user-metrics.csv',
'../metrics/slim/2016-05-10_03.24.56_slim_ml-100k_e3_tt-0.7_task-25_user-metrics.csv',
'../metrics/slim/2016-05-10_03.25.02_slim_ml-100k_e10_tt-0.7_task-31_user-metrics.csv',
'../metrics/slim/2016-05-10_03.25.02_slim_ml-100k_e3_tt-0.7_task-5_user-metrics.csv',
'../metrics/slim/2016-05-10_03.25.13_slim_ml-100k_e10_tt-0.7_task-39_user-metrics.csv',
'../metrics/slim/2016-05-10_03.25.18_slim_ml-100k_e8_tt-0.7_task-21_user-metrics.csv',
'../metrics/slim/2016-05-10_03.25.20_slim_ml-100k_e10_tt-0.7_task-47_user-metrics.csv',
'../metrics/slim/2016-05-10_03.25.20_slim_ml-100k_e8_tt-0.7_task-33_user-metrics.csv',
'../metrics/slim/2016-05-10_03.25.23_slim_ml-100k_e8_tt-0.7_task-15_user-metrics.csv',
'../metrics/slim/2016-05-10_03.25.27_slim_ml-100k_e10_tt-0.7_task-9_user-metrics.csv',
'../metrics/slim/2016-05-10_03.25.32_slim_ml-100k_e10_tt-0.7_task-7_user-metrics.csv',
'../metrics/slim/2016-05-10_03.25.36_slim_ml-100k_e5_tt-0.7_task-27_user-metrics.csv',
'../metrics/slim/2016-05-10_03.25.44_slim_ml-100k_e3_tt-0.7_task-17_user-metrics.csv',
'../metrics/slim/2016-05-10_03.25.47_slim_ml-100k_e8_tt-0.7_task-37_user-metrics.csv',
'../metrics/slim/2016-05-10_03.25.52_slim_ml-100k_e8_tt-0.7_task-35_user-metrics.csv',
'../metrics/slim/2016-05-10_03.42.54_slim_ml-100k_e10_tt-0.7_task-62_user-metrics.csv',
'../metrics/slim/2016-05-10_03.42.54_slim_ml-100k_e5_tt-0.7_task-48_user-metrics.csv',
'../metrics/slim/2016-05-10_03.43.12_slim_ml-100k_e3_tt-0.7_task-68_user-metrics.csv',
'../metrics/slim/2016-05-10_03.43.19_slim_ml-100k_e3_tt-0.7_task-50_user-metrics.csv',
'../metrics/slim/2016-05-10_03.43.20_slim_ml-100k_e10_tt-0.7_task-74_user-metrics.csv',
'../metrics/slim/2016-05-10_03.43.21_slim_ml-100k_e3_tt-0.7_task-52_user-metrics.csv',
'../metrics/slim/2016-05-10_03.43.40_slim_ml-100k_e8_tt-0.7_task-54_user-metrics.csv',
'../metrics/slim/2016-05-10_03.43.45_slim_ml-100k_e8_tt-0.7_task-56_user-metrics.csv',
'../metrics/slim/2016-05-10_03.43.47_slim_ml-100k_e3_tt-0.7_task-58_user-metrics.csv',
'../metrics/slim/2016-05-10_03.43.50_slim_ml-100k_e3_tt-0.7_task-60_user-metrics.csv',
'../metrics/slim/2016-05-10_03.43.59_slim_ml-100k_e8_tt-0.7_task-66_user-metrics.csv',
'../metrics/slim/2016-05-10_03.44.00_slim_ml-100k_e10_tt-0.7_task-72_user-metrics.csv',
'../metrics/slim/2016-05-10_03.44.03_slim_ml-100k_e3_tt-0.7_task-70_user-metrics.csv',
'../metrics/slim/2016-05-10_03.44.07_slim_ml-100k_e8_tt-0.7_task-90_user-metrics.csv',
'../metrics/slim/2016-05-10_03.44.09_slim_ml-100k_e10_tt-0.7_task-64_user-metrics.csv',
'../metrics/slim/2016-05-10_03.44.09_slim_ml-100k_e5_tt-0.7_task-78_user-metrics.csv',
'../metrics/slim/2016-05-10_03.44.15_slim_ml-100k_e8_tt-0.7_task-80_user-metrics.csv',
'../metrics/slim/2016-05-10_03.44.19_slim_ml-100k_e3_tt-0.7_task-86_user-metrics.csv',
'../metrics/slim/2016-05-10_03.44.25_slim_ml-100k_e3_tt-0.7_task-88_user-metrics.csv',
'../metrics/slim/2016-05-10_03.44.26_slim_ml-100k_e10_tt-0.7_task-76_user-metrics.csv',
'../metrics/slim/2016-05-10_03.44.43_slim_ml-100k_e3_tt-0.7_task-84_user-metrics.csv',
'../metrics/slim/2016-05-10_03.44.48_slim_ml-100k_e3_tt-0.7_task-92_user-metrics.csv',
'../metrics/slim/2016-05-10_03.44.49_slim_ml-100k_e8_tt-0.7_task-82_user-metrics.csv',
'../metrics/slim/2016-05-10_03.45.00_slim_ml-100k_e8_tt-0.7_task-94_user-metrics.csv',
'../metrics/slim/2016-05-10_04.01.18_slim_ml-100k_e8_tt-0.7_task-63_user-metrics.csv',
'../metrics/slim/2016-05-10_04.01.29_slim_ml-100k_e5_tt-0.7_task-69_user-metrics.csv',
'../metrics/slim/2016-05-10_04.01.50_slim_ml-100k_e8_tt-0.7_task-51_user-metrics.csv',
'../metrics/slim/2016-05-10_04.01.51_slim_ml-100k_e8_tt-0.7_task-53_user-metrics.csv',
'../metrics/slim/2016-05-10_04.02.02_slim_ml-100k_e3_tt-0.7_task-49_user-metrics.csv',
'../metrics/slim/2016-05-10_04.02.05_slim_ml-100k_e10_tt-0.7_task-55_user-metrics.csv',
'../metrics/slim/2016-05-10_04.02.07_slim_ml-100k_e3_tt-0.7_task-65_user-metrics.csv',
'../metrics/slim/2016-05-10_04.02.10_slim_ml-100k_e8_tt-0.7_task-91_user-metrics.csv',
'../metrics/slim/2016-05-10_04.02.18_slim_ml-100k_e8_tt-0.7_task-61_user-metrics.csv',
'../metrics/slim/2016-05-10_04.02.31_slim_ml-100k_e10_tt-0.7_task-75_user-metrics.csv',
'../metrics/slim/2016-05-10_04.02.36_slim_ml-100k_e5_tt-0.7_task-73_user-metrics.csv',
'../metrics/slim/2016-05-10_04.02.43_slim_ml-100k_e5_tt-0.7_task-89_user-metrics.csv',
'../metrics/slim/2016-05-10_04.02.52_slim_ml-100k_e3_tt-0.7_task-81_user-metrics.csv',
'../metrics/slim/2016-05-10_04.02.59_slim_ml-100k_e10_tt-0.7_task-59_user-metrics.csv',
'../metrics/slim/2016-05-10_04.03.07_slim_ml-100k_e10_tt-0.7_task-57_user-metrics.csv',
'../metrics/slim/2016-05-10_04.03.07_slim_ml-100k_e10_tt-0.7_task-71_user-metrics.csv',
'../metrics/slim/2016-05-10_04.03.15_slim_ml-100k_e10_tt-0.7_task-67_user-metrics.csv',
'../metrics/slim/2016-05-10_04.03.27_slim_ml-100k_e3_tt-0.7_task-79_user-metrics.csv',
'../metrics/slim/2016-05-10_04.03.29_slim_ml-100k_e5_tt-0.7_task-87_user-metrics.csv',
'../metrics/slim/2016-05-10_04.03.40_slim_ml-100k_e10_tt-0.7_task-85_user-metrics.csv',
'../metrics/slim/2016-05-10_04.03.48_slim_ml-100k_e8_tt-0.7_task-83_user-metrics.csv',
'../metrics/slim/2016-05-10_04.03.51_slim_ml-100k_e5_tt-0.7_task-93_user-metrics.csv',
'../metrics/slim/2016-05-10_04.04.00_slim_ml-100k_e8_tt-0.7_task-77_user-metrics.csv',
'../metrics/slim/2016-05-10_04.04.05_slim_ml-100k_e10_tt-0.7_task-95_user-metrics.csv',
'../metrics/slim/2016-05-10_04.20.40_slim_ml-100k_e10_tt-0.7_task-96_user-metrics.csv',
'../metrics/slim/2016-05-10_04.20.43_slim_ml-100k_e10_tt-0.7_task-98_user-metrics.csv',
'../metrics/slim/2016-05-10_04.39.45_slim_ml-100k_e5_tt-0.7_task-99_user-metrics.csv',
'../metrics/slim/2016-05-10_04.39.46_slim_ml-100k_e3_tt-0.7_task-97_user-metrics.csv',
]

In [3]:
result_map = map(lambda p: (p, pd.read_csv(p)['auc'].mean()), results)
sorted(result_map, key=lambda t:t[1], reverse=True)

[('../metrics/slim/2016-05-10_03.05.37_slim_ml-100k_e3_tt-0.7_task-22_user-metrics.csv',
  0.8952496912902969),
 ('../metrics/slim/2016-05-10_03.23.59_slim_ml-100k_e3_tt-0.7_task-1_user-metrics.csv',
  0.8936368573662951),
 ('../metrics/slim/2016-05-10_03.43.12_slim_ml-100k_e3_tt-0.7_task-68_user-metrics.csv',
  0.8935932610198645),
 ('../metrics/slim/2016-05-10_03.24.32_slim_ml-100k_e3_tt-0.7_task-41_user-metrics.csv',
  0.8934808859317592),
 ('../metrics/slim/2016-05-10_03.05.55_slim_ml-100k_e3_tt-0.7_task-2_user-metrics.csv',
  0.8934596832935031),
 ('../metrics/slim/2016-05-10_04.01.50_slim_ml-100k_e8_tt-0.7_task-51_user-metrics.csv',
  0.8921389843030674),
 ('../metrics/slim/2016-05-10_04.01.51_slim_ml-100k_e8_tt-0.7_task-53_user-metrics.csv',
  0.8921129722785286),
 ('../metrics/slim/2016-05-10_04.01.29_slim_ml-100k_e5_tt-0.7_task-69_user-metrics.csv',
  0.891756110913556),
 ('../metrics/slim/2016-05-10_04.02.43_slim_ml-100k_e5_tt-0.7_task-89_user-metrics.csv',
  0.89174248413777

In [5]:
with open('../models/slim/2016-05-10_02.47.26_slim_ml-100k_e3_tt-0.7_task-22_config.json') as f:
    pp.pprint(json.loads(f.read()))

{   u'alpha': 0.0033,
    u'binarize': True,
    u'binarize_neg': 0,
    u'binarize_pos': 1,
    u'binarize_threshold': 1,
    u'eval_in_parallel': False,
    u'experiment_name': u'no-si_ml-100k_e3_tt-0.7_task-22',
    u'fit_intercept': True,
    u'hit_threshold': 4,
    u'ignore_negative_weights': True,
    u'l1_ratio': 0.9090909090909091,
    u'l1_reg': 0.003,
    u'l2_reg': 0.0003,
    u'nb_epochs': 3,
    u'pool_size': 2,
    u'precision_recall_at_n': 20,
    u'ratings_path': u'data/splits/ml-100k/ratings.csv',
    u'run_eval': True,
    u'run_movie_metrics': False,
    u'sparse_item': True,
    u'test': True,
    u'test_path': u'data/splits/ml-100k/sparse-item/0.7-test.csv',
    u'top_n_predictions': 100,
    u'train_path': u'data/splits/ml-100k/sparse-item/0.7-0.8-train.csv',
    u'train_test_split': 0.7,
    u'train_val_split': 0.8,
    u'val': True,
    u'val_path': u'data/splits/ml-100k/sparse-item/0.7-0.8-val.csv',
    u'verbose': 0}


In [6]:
with open('../models/slim/2016-05-10_03.05.24_slim_ml-100k_e3_tt-0.7_task-1_config.json') as f:
    pp.pprint(json.loads(f.read()))

{   u'alpha': 0.004,
    u'binarize': True,
    u'binarize_neg': 0,
    u'binarize_pos': 1,
    u'binarize_threshold': 1,
    u'eval_in_parallel': False,
    u'experiment_name': u'no-si_ml-100k_e3_tt-0.7_task-1',
    u'fit_intercept': True,
    u'hit_threshold': 4,
    u'ignore_negative_weights': True,
    u'l1_ratio': 0.25,
    u'l1_reg': 0.001,
    u'l2_reg': 0.003,
    u'nb_epochs': 3,
    u'pool_size': 2,
    u'precision_recall_at_n': 20,
    u'ratings_path': u'data/splits/ml-100k/ratings.csv',
    u'run_eval': True,
    u'run_movie_metrics': False,
    u'sparse_item': True,
    u'test': True,
    u'test_path': u'data/splits/ml-100k/sparse-item/0.7-test.csv',
    u'top_n_predictions': 100,
    u'train_path': u'data/splits/ml-100k/sparse-item/0.7-0.8-train.csv',
    u'train_test_split': 0.7,
    u'train_val_split': 0.8,
    u'val': True,
    u'val_path': u'data/splits/ml-100k/sparse-item/0.7-0.8-val.csv',
    u'verbose': 0}


In [7]:
with open('../models/slim/2016-05-10_03.25.06_slim_ml-100k_e3_tt-0.7_task-68_config.json') as f:
    pp.pprint(json.loads(f.read()))

{   u'alpha': 0.006,
    u'binarize': True,
    u'binarize_neg': 0,
    u'binarize_pos': 1,
    u'binarize_threshold': 1,
    u'eval_in_parallel': False,
    u'experiment_name': u'no-si_ml-100k_e3_tt-0.7_task-68',
    u'fit_intercept': False,
    u'hit_threshold': 4,
    u'ignore_negative_weights': True,
    u'l1_ratio': 0.5,
    u'l1_reg': 0.003,
    u'l2_reg': 0.003,
    u'nb_epochs': 3,
    u'pool_size': 2,
    u'precision_recall_at_n': 20,
    u'ratings_path': u'data/splits/ml-100k/ratings.csv',
    u'run_eval': True,
    u'run_movie_metrics': False,
    u'sparse_item': True,
    u'test': True,
    u'test_path': u'data/splits/ml-100k/sparse-item/0.7-test.csv',
    u'top_n_predictions': 100,
    u'train_path': u'data/splits/ml-100k/sparse-item/0.7-0.8-train.csv',
    u'train_test_split': 0.7,
    u'train_val_split': 0.8,
    u'val': True,
    u'val_path': u'data/splits/ml-100k/sparse-item/0.7-0.8-val.csv',
    u'verbose': 0}


## ML-1m - 0.2 train / 0.8 test

In [8]:
results = [
'../metrics/slim/2016-05-12_21.03.23_slim_ml-1m_e3_tt-0.2_task-4_user-metrics.csv',
'../metrics/slim/2016-05-12_21.03.26_slim_ml-1m_e3_tt-0.2_task-13_user-metrics.csv',
'../metrics/slim/2016-05-12_21.03.30_slim_ml-1m_e3_tt-0.2_task-12_user-metrics.csv',
'../metrics/slim/2016-05-12_21.03.31_slim_ml-1m_e3_tt-0.2_task-16_user-metrics.csv',
'../metrics/slim/2016-05-12_21.03.34_slim_ml-1m_e3_tt-0.2_task-6_user-metrics.csv',
'../metrics/slim/2016-05-12_21.03.40_slim_ml-1m_e5_tt-0.2_task-9_user-metrics.csv',
'../metrics/slim/2016-05-12_21.03.50_slim_ml-1m_e5_tt-0.2_task-10_user-metrics.csv',
'../metrics/slim/2016-05-12_21.03.52_slim_ml-1m_e5_tt-0.2_task-14_user-metrics.csv',
'../metrics/slim/2016-05-12_21.04.02_slim_ml-1m_e5_tt-0.2_task-3_user-metrics.csv',
'../metrics/slim/2016-05-12_21.04.04_slim_ml-1m_e5_tt-0.2_task-22_user-metrics.csv',
'../metrics/slim/2016-05-12_21.04.13_slim_ml-1m_e5_tt-0.2_task-2_user-metrics.csv',
'../metrics/slim/2016-05-12_21.04.22_slim_ml-1m_e5_tt-0.2_task-15_user-metrics.csv',
'../metrics/slim/2016-05-12_21.05.01_slim_ml-1m_e8_tt-0.2_task-7_user-metrics.csv',
'../metrics/slim/2016-05-12_21.05.02_slim_ml-1m_e8_tt-0.2_task-20_user-metrics.csv',
'../metrics/slim/2016-05-12_21.05.05_slim_ml-1m_e8_tt-0.2_task-8_user-metrics.csv',
'../metrics/slim/2016-05-12_21.05.06_slim_ml-1m_e8_tt-0.2_task-18_user-metrics.csv',
'../metrics/slim/2016-05-12_21.05.11_slim_ml-1m_e10_tt-0.2_task-23_user-metrics.csv',
'../metrics/slim/2016-05-12_21.05.12_slim_ml-1m_e10_tt-0.2_task-1_user-metrics.csv',
'../metrics/slim/2016-05-12_21.05.22_slim_ml-1m_e8_tt-0.2_task-5_user-metrics.csv',
'../metrics/slim/2016-05-12_21.05.24_slim_ml-1m_e8_tt-0.2_task-19_user-metrics.csv',
'../metrics/slim/2016-05-12_21.05.24_slim_ml-1m_e10_tt-0.2_task-21_user-metrics.csv',
'../metrics/slim/2016-05-12_21.05.35_slim_ml-1m_e10_tt-0.2_task-11_user-metrics.csv',
'../metrics/slim/2016-05-12_21.05.57_slim_ml-1m_e10_tt-0.2_task-0_user-metrics.csv',
'../metrics/slim/2016-05-12_21.05.57_slim_ml-1m_e10_tt-0.2_task-17_user-metrics.csv',
'../metrics/slim/2016-05-12_21.20.51_slim_ml-1m_e3_tt-0.2_user-metrics.csv',
]

In [9]:
result_map = map(lambda p: (p, pd.read_csv(p)['auc'].mean()), results)
sorted(result_map, key=lambda t:t[1], reverse=True)

[('../metrics/slim/2016-05-12_21.03.31_slim_ml-1m_e3_tt-0.2_task-16_user-metrics.csv',
  0.8507230313839607),
 ('../metrics/slim/2016-05-12_21.20.51_slim_ml-1m_e3_tt-0.2_user-metrics.csv',
  0.8487254816158176),
 ('../metrics/slim/2016-05-12_21.05.06_slim_ml-1m_e8_tt-0.2_task-18_user-metrics.csv',
  0.8425732259742906),
 ('../metrics/slim/2016-05-12_21.05.24_slim_ml-1m_e10_tt-0.2_task-21_user-metrics.csv',
  0.8413169474062631),
 ('../metrics/slim/2016-05-12_21.05.12_slim_ml-1m_e10_tt-0.2_task-1_user-metrics.csv',
  0.8387943528011079),
 ('../metrics/slim/2016-05-12_21.03.30_slim_ml-1m_e3_tt-0.2_task-12_user-metrics.csv',
  0.8387208119342275),
 ('../metrics/slim/2016-05-12_21.03.26_slim_ml-1m_e3_tt-0.2_task-13_user-metrics.csv',
  0.8387186997100226),
 ('../metrics/slim/2016-05-12_21.04.04_slim_ml-1m_e5_tt-0.2_task-22_user-metrics.csv',
  0.8354776730842605),
 ('../metrics/slim/2016-05-12_21.04.02_slim_ml-1m_e5_tt-0.2_task-3_user-metrics.csv',
  0.8354776431488781),
 ('../metrics/slim

In [10]:
with open('../models/slim/2016-05-12_21.03.31_slim_ml-1m_e3_tt-0.2_task-16_config.json') as f:
    pp.pprint(json.loads(f.read()))

{   u'alpha': 0.002,
    u'binarize': True,
    u'binarize_neg': 0,
    u'binarize_pos': 1,
    u'binarize_threshold': 1,
    u'eval_in_parallel': False,
    u'experiment_name': u'slim_ml-1m_e3_tt-0.2_task-16',
    u'fit_intercept': True,
    u'hit_threshold': 4,
    u'ignore_negative_weights': False,
    u'l1_ratio': 0.5,
    u'l1_reg': 0.001,
    u'l2_reg': 0.001,
    u'metrics_save_dir': u'metrics/slim',
    u'model_save_dir': u'models/slim',
    u'nb_epochs': 3,
    u'pool_size': 2,
    u'precision_recall_at_n': 20,
    u'ratings_path': u'data/splits/ml-1m/ratings.csv',
    u'run_eval': True,
    u'run_movie_metrics': False,
    u'sparse_item': True,
    u'test': True,
    u'test_path': u'data/splits/ml-1m/sparse-item/0.2-test.csv',
    u'top_n_predictions': 100,
    u'train_path': u'data/splits/ml-1m/sparse-item/0.2-train.csv',
    u'train_test_split': 0.2,
    u'val': False,
    u'verbose': 0}


In [11]:
with open('../models/slim/2016-05-12_21.20.51_slim_ml-1m_e3_tt-0.2_config.json') as f:
    pp.pprint(json.loads(f.read()))

{   u'alpha': 0.0011,
    u'binarize': True,
    u'binarize_neg': 0,
    u'binarize_pos': 1,
    u'binarize_threshold': 1,
    u'debug_eval': False,
    u'eval_in_parallel': True,
    u'experiment_name': u'slim_ml-1m_e3_tt-0.2',
    u'fit_intercept': True,
    u'hit_threshold': 4,
    u'ignore_negative_weights': False,
    u'l1_ratio': 0.9090909090909091,
    u'l1_reg': 0.001,
    u'l2_reg': 0.0001,
    u'metrics_save_dir': u'metrics/slim',
    u'model_save_dir': u'models/slim',
    u'nb_epochs': 5,
    u'pool_size': 4,
    u'precision_recall_at_n': 20,
    u'ratings_path': u'data/splits/ml-1m/ratings.csv',
    u'run_eval': True,
    u'run_movie_metrics': False,
    u'sparse_item': True,
    u'test': True,
    u'test_path': u'data/splits/ml-1m/sparse-item/0.2-test.csv',
    u'top_n_predictions': 100,
    u'train_path': u'data/splits/ml-1m/sparse-item/0.2-train.csv',
    u'train_test_split': 0.2,
    u'val': False,
    u'verbose': 1}


In [12]:
with open('../models/slim/2016-05-12_21.05.06_slim_ml-1m_e8_tt-0.2_task-18_config.json') as f:
    pp.pprint(json.loads(f.read()))

{   u'alpha': 0.0013,
    u'binarize': True,
    u'binarize_neg': 0,
    u'binarize_pos': 1,
    u'binarize_threshold': 1,
    u'eval_in_parallel': False,
    u'experiment_name': u'slim_ml-1m_e8_tt-0.2_task-18',
    u'fit_intercept': False,
    u'hit_threshold': 4,
    u'ignore_negative_weights': True,
    u'l1_ratio': 0.23076923076923075,
    u'l1_reg': 0.0003,
    u'l2_reg': 0.001,
    u'metrics_save_dir': u'metrics/slim',
    u'model_save_dir': u'models/slim',
    u'nb_epochs': 8,
    u'pool_size': 2,
    u'precision_recall_at_n': 20,
    u'ratings_path': u'data/splits/ml-1m/ratings.csv',
    u'run_eval': True,
    u'run_movie_metrics': False,
    u'sparse_item': True,
    u'test': True,
    u'test_path': u'data/splits/ml-1m/sparse-item/0.2-test.csv',
    u'top_n_predictions': 100,
    u'train_path': u'data/splits/ml-1m/sparse-item/0.2-train.csv',
    u'train_test_split': 0.2,
    u'val': False,
    u'verbose': 0}
